We need to get from mapped bam file to single fast5s:

Input is:
* mapped bam file.
* multifast5 of fastqs mapped to the reference

Output:
* folder with single fast5s that are mapped

What we need to do:
* get ID of all mapped fastqs
* extract them in batches
* make them single fast5s

In [25]:
import os

### INPUTS

### *****Define directories before running on different sampes*****

In [26]:
###define input directories here 
BAMIN_DIR = '../../analyses/mapping/infected_leaves/infected_leaves_4/'
FAST5IN_DIR = '../../analyses/single_fast5s/infected_leaves/infected_leaves_4_fast5_single_fast5/'
OUT_DIR = '../../analyses/single_fast5s/infected_leaves/infected_leaves_4_mapped_single_fast5/'


### *****Define directories before running on different sampes*****

In [27]:
BAMIN_DIR = os.path.abspath(BAMIN_DIR)
FAST5IN_DIR = os.path.abspath(FAST5IN_DIR)
OUT_DIR = os.path.abspath(OUT_DIR)

In [28]:
##catch the bam based on naming convention
inbam_fn = [os.path.join(BAMIN_DIR, x) for x in os.listdir(BAMIN_DIR) if x.endswith('sorted.bam')][0]

In [29]:
##generated the read ID list
mappedids_fn = inbam_fn.replace('.bam', '.mappedids.txt')
#!samtools  view -F 4  {inbam_fn} | cut -f 1 | sort | uniq > {mappedids_fn}

In [30]:
!head {mappedids_fn}

0000d07e-32f4-49ab-8693-dd74fa628096
00020ccf-d7e9-44e2-9496-55cf0d8e1852
00021fa7-9a0f-4712-a2b5-26dc2a3abe64
0002c598-c6f2-4e65-865e-9c263de899ad
0003f8e5-75c5-4d95-b546-2f56e25efd3a
00040ef8-a126-4df8-8387-f14c8ee8724a
0004f268-c36b-4023-8fc9-7a66c7fef7d9
00054516-f1f2-4c6e-8d57-3dde6049e51f
00059b74-4672-42f3-92f5-1f4eb2cbf05f
00060d0e-b77d-407d-9695-8490a8c5c18e


In [31]:
### generate a temporary folder that holds the batch fast5s
TMPOUT_DIR = os.path.join(OUT_DIR, 'tmp')
if not os.path.exists(TMPOUT_DIR):
    os.makedirs(TMPOUT_DIR)

In [32]:
#read in the mapped reads ids
mapped_reads = []
with open(mappedids_fn) as fh:
    for line in fh:
        mapped_reads.append(line.rstrip())

In [33]:
mapped_reads = set(mapped_reads)

In [10]:
###go from multies fast5 of mapped reads to single fast5s
!multi_to_single_fast5 -i {FAST5IN_DIR} -s {TMPOUT_DIR} -t 10 --recursive
!touch {OUT_DIR}/single_fast5.done

^C25 of 1942|                                         |  1% ETA:  1 day, 4:27:17
Traceback (most recent call last):
  File "/home/jamila/anaconda3/bin/multi_to_single_fast5", line 10, in <module>
    sys.exit(main())
  File "/home/jamila/anaconda3/lib/python3.7/site-packages/ont_fast5_api/conversion_tools/multi_to_single_fast5.py", line 104, in main
    args.recursive, follow_symlinks=not args.ignore_symlinks)
  File "/home/jamila/anaconda3/lib/python3.7/site-packages/ont_fast5_api/conversion_tools/multi_to_single_fast5.py", line 40, in batch_convert_multi_files_to_single
    pool.join()
  File "/home/jamila/anaconda3/lib/python3.7/multiprocessing/pool.py", line 556, in join
    self._worker_handler.join()
  File "/home/jamila/anaconda3/lib/python3.7/threading.py", line 1044, in join
    self._wait_for_tstate_lock()
  File "/home/jamila/anaconda3/lib/python3.7/threading.py", line 1060, in _wait_for_tstate_lock
    elif lock.acquire(block, timeout):
KeyboardInterrupt


In [34]:
#move fast5s you want from tmp to out dir
count = 0
for directory in os.listdir(TMPOUT_DIR):
    directory = os.path.join(TMPOUT_DIR, directory)
    #check if path is directory
    if os.path.isdir(directory):
        print(directory)
        fast5s = [fn for fn in os.listdir(directory)]
        for fast5_file in fast5s:
            if fast5_file.rstrip('.fast5') in mapped_reads:
                count = count + 1
                #move the files by renaming absolute path
                old_fn = os.path.join(directory, fast5_file)
                new_fn = os.path.join(OUT_DIR, fast5_file)
                os.replace(old_fn, new_fn)
        #count = count + len(fast5s)
        print(count)
        
    #print(os.path.isfile(x))

In [35]:
count == len(mapped_reads)

False

In [36]:
for x in os.listdir(OUT_DIR):
    if not x.endswith('.fast5'):
        print(x)

tmp


In [17]:
#remove tmp folder
!rm -r {TMPOUT_DIR}


In [37]:
len(mapped_reads)

93264